1. [Introduction: Post-processing with Reject Option Classification](#introduction)
1. [Load all necessary packages](#paragraph2)
1. [Create the dataframe from a csv file](#paragraph3)
1. [Create an aif360 dataset from the dataframe](#paragraph4)
    1. [Convert the dataframe into an aif360 dataset](#subparagraph4.1)
    1. [Split into training, validation, and testing datasets](#subparagraph4.2)
    1. [Display properties of the datasets](#subparagraph4.3)
    1. [Compute raw fairness for datasetss - age category as protected attribute, privileged group 1 (25+ year old) and unprivileged group 0 (younger than 25)](#subparagraph4.4)
    1. [Scale training data and use that scaler to transform the validation and testing datasets](#subparagraph4.5)
1. [Logistic Regression (LR)](#paragraph5)
    1. [Build a LR model on training data](#subparagraph5.1)
    1. [Fairness metrics for LR model, before correcting for bias](#subparagraph5.2)
    1. [Post-processing the LR  model with ROC, optimizing Disparate Impact](#subparagraph5.3)
    1. [Fairness metrics for LR model, after correcting for bias (with optimized Disparate Impact)](#subparagraph5.4)
    1. [Post-processing the LR  model with ROC, optimizing Difference in Odds](#subparagraph5.5)
    1. [Fairness metrics for LR model, after correcting for bias (with optimized Difference in Odds)](#subparagraph5.6)
    1. [Post-processing the LR model with EqOddsPostprocessing](#subparagraph5.7)
    1. [Fairness metrics for LR model, after correcting for bias (with optimized Equalized Odds)](#subparagraph5.8)
1. [Extreme Gradient Boosting (XGB)](#paragraph6)
    1. [Build an XGB model on training data](#subparagraph6.1)
    1. [Fairness metrics for XGB model, before correcting for bias](#subparagraph6.2)
    1. [Post-processing the XGB  model with ROC, optimizing Disparate Impact](#subparagraph6.3)
    1. [Fairness metrics for XGB model, after correcting for bias (with optimized Disparate Impact)](#subparagraph6.4)
    1. [Post-processing the XGB  model with ROC, optimizing Difference in Odds](#subparagraph6.5)
    1. [Fairness metrics for XGB model, after correcting for bias (with optimized Difference in Odds)](#subparagraph6.6))
    1. [Post-processing the XGB model with EqOddsPostprocessing](#subparagraph6.7)
    1. [Fairness metrics for XGB model, after correcting for bias (with optimized Equalized Odds)](#subparagraph6.8)

* https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
* https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
* https://scikit-learn.org/stable/modules/model_evaluation.html

# 1. Introduction: Post-processing with Reject Option Classification <a name='introduction'></a>

#### This notebook uses the Reject Option Classification (ROC) post-processing algorithm for bias mitigation. The steps are:
* All necessary packages are imported
* The csv file is imported to create a pandas dataframe
* An aif360 dataset is created from the pandas dataframe. This step is critical for the analysis because the ROC algorithm does not operate on a pandas dataframe but on an aif360 dataset. The aif360 dataset is also split into a training set, validation set, and testing set (70%, 15%, and 15% of the dataset, respectively)
* A Logistic Regression model is built, optimizing the Balanced Accuracy metric. The predictions from this model are processed by ROC so that (more) fair predictions are obtained ("fair" as determined by the Disparate Impact metric)
* An Extreme Gradient Boosting is built, with tuning the parameters by cross validation and optimizing the Balanced Accuracy metric. The predictions from this model are processed by ROC so that (more) fair predictions are obtained ("fair, again according the Disparate Impact metric).

#### References:
* <a href="https://towardsdatascience.com/reducing-ai-bias-with-rejection-option-based-classification-54fefdb53c2">Introduction to fairness with post-processing</a>
* <a href="https://aif360.mybluemix.net">IBM aif3360 library/a>

## 2. Load all necessary packages <a name="paragraph2"></a>

!pip install 

missingno_found = !pip list | grep missingno != ""
if not missingno_found:
    !pip install missingno

In [1]:
# install and import IBM's fairness package
aif360_found = !pip list | grep aif360 != ""
if not aif360_found:
    !pip install aif360
    
import aif360

In [2]:
from IPython.display import Markdown, display
import time
import pickle

import matplotlib.pyplot as plt
%matplotlib inline

from ipywidgets import interactive, FloatSlider


#import sys
#sys.path.append("../")

from tqdm import tqdm

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
#import missingno as msno

import warnings
warnings.filterwarnings('ignore')

In [3]:
# import aif360 datasets and BinaryLabelDataset
import aif360.datasets

from aif360.datasets import BinaryLabelDataset
#from aif360.datasets import AdultDataset, GermanDataset, CompasDataset

In [4]:
# import fairness metrics

while True:
    try:
        import aif360.metrics
    except:
         continue
    else:
         break

pip install 'aif360[AdversarialDebiasing]'


In [5]:
from aif360.algorithms.postprocessing.reject_option_classification\
        import RejectOptionClassification

from aif360.metrics import ClassificationMetric

while True:
    try:
        from aif360.metrics import BinaryLabelDatasetMetric
    except:
         continue
    else:
         break

In [6]:
#import Equalized Post Processing module

while True:
    try:
        from aif360.algorithms.postprocessing import EqOddsPostprocessing
    except:
         continue
    else:
         break

In [7]:
# import StandardScaler to transform to mean 0, standard deviation 1
from sklearn.preprocessing import StandardScaler

# import machine learning algorithms
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import accuracy_score, balanced_accuracy_score

In [8]:
def compute_classification_metrics (dataset_orig_trn, dataset_pred_trn,
                                    dataset_orig_vld, dataset_pred_vld,
                                    dataset_orig_tst, dataset_pred_tst):
    """
    Computes classification metrics for training, validation and testing data, 
    using the aif360 ClassificationMetric function
    
    Input: 
    - dataset_orig: original dataset with labels
    - dataset_pred: dataset with predictions (possibly the transformed predictions, transformed by postprocessing)
    
    Returns:
    - accuracy
    - balanced accuracy
    - absolute difference in odds
    - disparate impact
    """
    
    metrics =  [-1] * 15
    
    cm_train = ClassificationMetric(dataset_orig_trn,
                                    dataset_pred_trn,
                                    unprivileged_groups=unprivileged_groups,
                                    privileged_groups=privileged_groups)
    
    metrics [0]  = cm_train.accuracy()
    metrics [1]  = (cm_train.true_positive_rate() + cm_train.true_negative_rate())/ 2
    metrics [2]  = cm_train.average_abs_odds_difference()
    metrics [3]  = cm_train.average_odds_difference()
    metrics [4]  = cm_train.disparate_impact()
    
    
    cm_valid = ClassificationMetric(dataset_orig_vld,
                              dataset_pred_vld,
                              unprivileged_groups=unprivileged_groups,
                              privileged_groups=privileged_groups)
    
    metrics [5] = cm_valid.accuracy()
    metrics [6] = (cm_valid.true_positive_rate() + cm_valid.true_negative_rate())/ 2
    metrics [7] = cm_valid.average_abs_odds_difference()
    metrics [8] = cm_valid.average_odds_difference()
    metrics [9] = cm_valid.disparate_impact()
    
    
    cm_test = ClassificationMetric(dataset_orig_tst,
                              dataset_pred_tst,
                              unprivileged_groups=unprivileged_groups,
                              privileged_groups=privileged_groups)
    
    metrics [10] = cm_test.accuracy()
    metrics [11] = (cm_test.true_positive_rate() + cm_test.true_negative_rate())/ 2
    metrics [12] = cm_test.average_abs_odds_difference()
    metrics [13] = cm_test.average_odds_difference()
    metrics [14] = cm_test.disparate_impact()
    
    return metrics

In [9]:
def print_classification_metrics(metrics):
    """
    Prints the classification metrics
    """
    
    for i in range (len(metrics)):
        if i <= 4:
            if i == 0:
                display(Markdown("##### Results for the training dataset\n"))
                print ("Accuracy:\t\t\t", round (metrics[0], 3))
            if i == 1:
                print ("Balanced Accuracy:\t\t", round (metrics[1], 3))
            if i == 2:
                print("Absolute difference in odds:\t", round(metrics[2], 3))
            if i == 3:
                print("Difference in odds:\t\t", round(metrics[3], 3)) 
            if i == 4:
                print("Disparate impact:\t\t", round(metrics[4], 3))
        elif i <= 9:
            if i == 5:
                display(Markdown("##### Results for the validation dataset\n"))
                print ("Accuracy:\t\t\t", round (metrics[5], 3))
            if i == 6:
                print ("Balanced Accuracy:\t\t", round (metrics[6], 3))
            if i == 7:
                print("Absolute difference in odds:\t", round(metrics[7], 3))
            if i == 8:
                print("Difference in odds:\t\t", round(metrics[8], 3))
            if i == 9:
                print("Disparate impact:\t\t", round(metrics[9], 3))
        else:
            if i == 10:
                display(Markdown("##### Results for the testing dataset\n"))
                print ("Accuracy:\t\t\t", round (metrics[10], 3))
            if i == 11:
                print ("Balanced Accuracy:\t\t", round (metrics[11], 3))
            if i == 12:
                print("Absolute difference in odds:\t", round(metrics[12], 3))
            if i == 13:
                print("Difference in odds:\t\t", round(metrics[13], 3))
            if i == 14:
                print("Disparate impact:\t\t", round(metrics[14], 3))

## 3. Create the dataframe from a csv file<a name="paragraph3"></a>

In [10]:
my_path = "C:/Users/Gebruiker/Lotte_Thesis/"
my_filename = "german_credit_data_prepared.csv"
my_path_filename = my_path + my_filename
df_new = pd.read_csv(my_path_filename)
#cols_to_drop = ["gender_male", "marital_status_divorced/separated", "marital_status_divorced/separated/married",
#                "marital_status_married/widowed", "marital_status_single"]
#df_new.drop(cols_to_drop, axis=1, inplace=True)
df_new.head()

,Duration in month,Credit amount,Installment rate in percentage of disposable income,Present residence since,Number of existing credits at this bank,Number of people being liable to provide maintenance for,gender_male,marital_status_divorced/separated,marital_status_divorced/separated/married,marital_status_married/widowed,...,Housing_rent,Job_management/ highly qualified employee,Job_skilled employee / official,Job_unemployed/ unskilled - non-resident,Job_unskilled - resident,Telephone_none,Telephone_yes,foreign worker_no,foreign worker_yes,good_risk
0,6,1169,4,4,2,1,1,0,0,0,...,0,0,1,0,0,0,1,0,1,1
1,48,5951,2,2,1,1,0,0,1,0,...,0,0,1,0,0,1,0,0,1,0
2,12,2096,2,3,1,2,1,0,0,0,...,0,0,0,0,1,1,0,0,1,1
3,42,7882,2,4,1,2,1,0,0,0,...,0,0,1,0,0,1,0,0,1,1
4,24,4870,3,4,2,2,1,0,0,0,...,0,0,1,0,0,1,0,0,1,0


## 4. Create an aif360 dataset from the dataframe <a name="paragraph4"></a>

### A. Convert the dataframe into an aif360 dataset <a name="subparagraph4.1"></a>

In [11]:
# aif360 algorithms work on a "aif360" dataset, so convert the dataframe into an aif360 dataset

dataset_orig = aif360.datasets.BinaryLabelDataset(df=df_new,
                                                  label_names=['good_risk'],
                                                  protected_attribute_names=['age_25plus'],
                                                  favorable_label = 1.0,
                                                  unfavorable_label=0.0,
                                                  privileged_protected_attributes =  [[1]],
                                                  unprivileged_protected_attributes =  [[0]]
                                                 )       

### B. Split into training, validation, and testing datasets<a name="subparagraph4.2"></a>

In [12]:
# Get the dataset and split into train and test
dataset_orig_train, dataset_orig_vt = dataset_orig.split([0.7], shuffle=True)
dataset_orig_valid, dataset_orig_test = dataset_orig_vt.split([0.7], shuffle=True)

### C. Display properties of the datasets  <a name="subparagraph4.3"></a>

In [13]:
# print out some labels, names, etc.
display(Markdown("#### Training Dataset shape"))
print(dataset_orig_train.features.shape)
display(Markdown("#### Validation Dataset shape"))
print(dataset_orig_valid.features.shape)
display(Markdown("#### Testing Dataset shape"))
print(dataset_orig_test.features.shape)
display(Markdown("#### Favorable and unfavorable labels"))
print(dataset_orig_train.favorable_label, dataset_orig_train.unfavorable_label)
display(Markdown("#### Protected attribute names"))
print(dataset_orig_train.protected_attribute_names)
display(Markdown("#### Privileged and unprivileged protected attribute values"))
print(dataset_orig_train.privileged_protected_attributes, 
      dataset_orig_train.unprivileged_protected_attributes)
display(Markdown("#### Dataset feature names"))
print(dataset_orig_train.feature_names)

#### Training Dataset shape

(700, 62)


#### Validation Dataset shape

(210, 62)


#### Testing Dataset shape

...

['age_25plus']


#### Privileged and unprivileged protected attribute values

[[1]] [[0]]


#### Dataset feature names

['Duration in month', 'Credit amount', 'Installment rate in percentage of disposable income', 'Present residence since', 'Number of existing credits at this bank', 'Number of people being liable to provide maintenance for', 'gender_male', 'marital_status_divorced/separated', 'marital_status_divorced/separated/married', 'marital_status_married/widowed', 'marital_status_single', 'age_25plus', 'Status of existing checking account_0 <= <200 DM', 'Status of existing checking account_<0 DM', 'Status of existing checking account_>= 200 DM ', 'Status of existing checking account_no checking account', 'Credit history_all credits at this bank paid back duly', 'Credit history_critical account', 'Credit history_delay in paying off', 'Credit history_existing credits paid back duly till now', 'Credit history_no credits taken', 'Purpose_business', 'Purpose_car (new)', 'Purpose_car (used)', 'Purpose_domestic appliances', 'Purpose_education', 'Purpose_furniture/equipment', 'Purpose_others', 'Purpose_ra

### D. Compute raw fairness for datasetss - age category as protected attribute, privileged group 1 (25+ year old) and unprivileged group 0 (younger than 25)<a name="subparagraph4.4"></a>

* "raw" fairness: fairness without a model; here, just look at the difference between the proportion good risk of the unprivileged group and the proportion good risk of the privileged group.
* The difference is computed as: proportion good risk unprivileged group - proportion good risk privileged group, so negative values indicate bias

In [14]:
# Define privileged and unprivileged groups
privileged_groups =   [{'age_25plus': 1}]
unprivileged_groups = [{'age_25plus': 0}]

In [15]:
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train,
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Difference in mean outcomes between unprivileged and privileged group"))
print("Training dataset:\t %.3f" % metric_orig_train.mean_difference())

metric_orig_valid = BinaryLabelDatasetMetric(dataset_orig_valid, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Validation dataset:\t %.3f" % metric_orig_valid.mean_difference())

metric_orig_test = BinaryLabelDatasetMetric(dataset_orig_test,
                                            unprivileged_groups=unprivileged_groups,
                                            privileged_groups=privileged_groups)
print("Testing dataset:\t %.3f" % metric_orig_test.mean_difference())

#### Difference in mean outcomes between unprivileged and privileged group

Training dataset:	 -0.153
Validation dataset:	 0.034
Testing dataset:	 -0.348


* For training, validation, and testing set, those who are 25+ have a higher probability to be a good risk. 

### E. Scale training data and use that scaler to transform the validation and testing datasets<a name="subparagraph4.5"></a>

In [16]:
# Create training dataset by standardizing columns
scale_orig = StandardScaler()

# Fit and scale features in the training set
X_train = scale_orig.fit_transform(dataset_orig_train.features)
y_train = dataset_orig_train.labels.ravel()

# Scale validation data
X_valid = scale_orig.transform(dataset_orig_valid.features)
#y_valid = dataset_orig_valid_pred.labels

# Scale test data
X_test = scale_orig.transform(dataset_orig_test.features)
#y_test = dataset_orig_test_pred.labels

## 5. Logistic Regression (LR) <a name="paragraph5"></a>

In [17]:
# Placeholders for the predictions on the original datasets
dataset_orig_train_pred = dataset_orig_train.copy(deepcopy=True)
dataset_orig_valid_pred = dataset_orig_valid.copy(deepcopy=True)
dataset_orig_test_pred  = dataset_orig_test.copy(deepcopy=True)

### A. Build a LR model on training data<a name="subparagraph5.1"></a>

In [18]:
# Run Logistic Regression and save the predictions to the training, validation and testing datasets
lrmod = LogisticRegression()
lrmod.fit(X_train, y_train)

LogisticRegression()

In [19]:
# Print the coefficient of age_25plus, to see if there is bias
lr_coeff = lrmod.coef_ [0]

i = 0        
for col in df_new.columns:
    if col  == "age_25plus":
        print ("Coefficient for", col, ":\t %.3f" %  lr_coeff[i])
    i += 1

Coefficient for age_25plus :	 0.209


* The coefficient is positive, so 25+ year old people have a higher probability to be classified as good risk

### B. Fairness metrics for LR model, before correcting for bias<a name="subparagraph5.2"></a>

In [20]:
# The aif360 ClassificationMetric function only works for aif360 datasets, so add the predictions (probabilities and
# and predicted labels) to the training, validation, and testing dtaasets (which are BinaryLabelDatasets)

fav_idx = np.where(lrmod.classes_ == dataset_orig_train.favorable_label)[0][0]
#pos_ind = np.where(lrmod.classes_ == dataset_orig_train.favorable_label)[0][0]

# Prediction probs for training, validation and testing data
y_train_pred_prob = lrmod.predict_proba(X_train)[:,fav_idx]
y_valid_pred_prob = lrmod.predict_proba(X_valid)[:,fav_idx]
y_test_pred_prob  = lrmod.predict_proba(X_test)[:,fav_idx]

class_thresh = 0.5

dataset_orig_train_pred.scores = y_train_pred_prob.reshape(-1,1)
dataset_orig_valid_pred.scores = y_valid_pred_prob.reshape(-1,1)
dataset_orig_test_pred.scores  = y_test_pred_prob.reshape(-1,1)

y_train_pred = np.zeros_like(dataset_orig_train_pred.labels)
y_train_pred[y_train_pred_prob >= class_thresh] = dataset_orig_train_pred.favorable_label
y_train_pred[~(y_train_pred_prob >= class_thresh)] = dataset_orig_train_pred.unfavorable_label
dataset_orig_train_pred.labels = y_train_pred

y_valid_pred = np.zeros_like(dataset_orig_valid_pred.labels)
y_valid_pred[y_valid_pred_prob >= class_thresh] = dataset_orig_valid_pred.favorable_label
y_valid_pred[~(y_valid_pred_prob >= class_thresh)] = dataset_orig_valid_pred.unfavorable_label
dataset_orig_valid_pred.labels = y_valid_pred

y_test_pred = np.zeros_like(dataset_orig_test_pred.labels)
y_test_pred[y_test_pred_prob >= class_thresh] = dataset_orig_test_pred.favorable_label
y_test_pred[~(y_test_pred_prob >= class_thresh)] = dataset_orig_test_pred.unfavorable_label
dataset_orig_test_pred.labels = y_test_pred

In [21]:
display(Markdown("#### Results for Logistic Regression before correcting for bias by post-processing"))

lr_bef_class_metrics= compute_classification_metrics (dataset_orig_train, dataset_orig_train_pred,
                                                      dataset_orig_valid, dataset_orig_valid_pred,
                                                      dataset_orig_test,  dataset_orig_test_pred)

print_classification_metrics(lr_bef_class_metrics)

#### Results for Logistic Regression before correcting for bias by post-processing

##### Results for the training dataset


Accuracy:			 0.779
Balanced Accuracy:		 0.699
Absolute difference in odds:	 0.101
Difference in odds:		 -0.101
Disparate impact:		 0.786


##### Results for the validation dataset


Accuracy:			 0.762
Balanced Accuracy:		 0.683
Absolute difference in odds:	 0.207
Difference in odds:		 -0.207
Disparate impact:		 0.734


##### Results for the testing dataset


Accuracy:			 0.789
Balanced Accuracy:		 0.718
Absolute difference in odds:	 0.256
Difference in odds:		 -0.256
Disparate impact:		 0.525


### C. Post-processing the LR  model with ROC, optimizing Disparate Impact<a name="subparagraph5.3"></a>

In [22]:
# Placeholders for the predictions on the transformed datasets
dataset_transf_train_pred = dataset_orig_train.copy(deepcopy=True)
dataset_transf_valid_pred = dataset_orig_valid.copy(deepcopy=True)
dataset_transf_test_pred = dataset_orig_test.copy(deepcopy=True)

In [23]:
# Allowed options for the fairness metric which must be optimized for ROC are “Statistical parity difference”, 
# “Average odds difference”, and “Equal opportunity difference”.

In [24]:
# Determine the best threshold for classification on the validation set, with fairness constraint.
# For that, we can use aif360's Reject Option Classsification

ROC = RejectOptionClassification(unprivileged_groups=unprivileged_groups, 
                                 privileged_groups=privileged_groups, 
                                 low_class_thresh=0.01, high_class_thresh=0.99,
                                 num_class_thresh=100, num_ROC_margin=50,
                                 metric_name="Statistical parity difference",
                                 metric_ub=0.05, 
                                 metric_lb=-0.05)
ROC_di = ROC.fit(dataset_orig_valid, dataset_orig_valid_pred)

In [25]:
print("Optimal classification threshold (with fairness constraint) = %.3f" % ROC_di.classification_threshold)
print("Optimal ROC margin = %.3f" % ROC_di.ROC_margin)

Optimal classification threshold (with fairness constraint) = 0.495
Optimal ROC margin = 0.131


### D. Fairness metrics for LR model, after correcting for bias (with optimized Disparate Impact)<a name="subparagraph5.4"></a>

In [26]:
# Metrics for the transformed test set
dataset_transf_train_pred = ROC_di.predict(dataset_orig_train_pred) 
dataset_transf_valid_pred = ROC_di.predict(dataset_orig_valid_pred)
dataset_transf_test_pred  = ROC_di.predict(dataset_orig_test_pred)

In [27]:
display(Markdown("#### Results for Logistic Regression after correcting for bias (optimizing Disparate Impact)"))

lr_aft_class_metrics_di = compute_classification_metrics (dataset_orig_train, dataset_transf_train_pred,
                                                     dataset_orig_valid, dataset_transf_valid_pred,
                                                     dataset_orig_test,  dataset_transf_test_pred)

print_classification_metrics(lr_aft_class_metrics_di)

#### Results for Logistic Regression after correcting for bias (optimizing Disparate Impact)

##### Results for the training dataset


Accuracy:			 0.776
Balanced Accuracy:		 0.729
Absolute difference in odds:	 0.109
Difference in odds:		 0.106
Disparate impact:		 1.019


##### Results for the validation dataset


Accuracy:			 0.795
Balanced Accuracy:		 0.744
Absolute difference in odds:	 0.254
Difference in odds:		 0.107
Disparate impact:		 1.024


##### Results for the testing dataset


Accuracy:			 0.722
Balanced Accuracy:		 0.672
Absolute difference in odds:	 0.109
Difference in odds:		 -0.021
Disparate impact:		 0.804


In [28]:
# Testing: Check if the Disparate Impact has not become worse
#assert lr_aft_class_metrics_di[14] >= lr_bef_class_metrics[14]

### E. Post-processing the LR model with ROC, optimizing Difference in Odds<a name="subparagraph5.5"></a>

In [29]:
# Placeholders for the predictions on the transformed datasets
dataset_transf_train_pred = dataset_orig_train.copy(deepcopy=True)
dataset_transf_valid_pred = dataset_orig_valid.copy(deepcopy=True)
dataset_transf_test_pred = dataset_orig_test.copy(deepcopy=True)

In [30]:
# Determine the best threshold for classification on the validation set, with fairness constraint.
# For that, we can use aif360's Reject Option Classsification

# Allowed options for the ROC fairness metric which must be optimized are “Statistical parity difference”, “Average odds difference”, 
# and “Equal opportunity difference”.

ROC = RejectOptionClassification(unprivileged_groups=unprivileged_groups, 
                                 privileged_groups=privileged_groups, 
                                 low_class_thresh=0.01, high_class_thresh=0.99,
                                 num_class_thresh=100, num_ROC_margin=50,
                                 metric_name= "Average odds difference", # "Equal opportunity difference",
                                 metric_ub=0.05, 
                                 metric_lb=-0.05)
ROC_aod = ROC.fit(dataset_orig_valid, dataset_orig_valid_pred)

In [31]:
print("Optimal classification threshold (with fairness constraint) = %.3f" % ROC_aod.classification_threshold)
print("Optimal ROC margin = %.3f" % ROC_aod.ROC_margin)

Optimal classification threshold (with fairness constraint) = 0.554
Optimal ROC margin = 0.073


### F. Fairness metrics for LR model, after correcting for bias (with optimized Difference in Odds)<a name="subparagraph5.6"></a>

In [32]:
# Metrics for the transformed test set
dataset_transf_train_pred = ROC_aod.predict(dataset_orig_train_pred) 
dataset_transf_valid_pred = ROC_aod.predict(dataset_orig_valid_pred)
dataset_transf_test_pred  = ROC_aod.predict(dataset_orig_test_pred)

In [33]:
display(Markdown("#### Results for Logistic Regression after correcting for bias (optimizing Difference in Odds)"))

lr_aft_class_metrics_aod = compute_classification_metrics (dataset_orig_train, dataset_transf_train_pred,
                                                           dataset_orig_valid, dataset_transf_valid_pred,
                                                           dataset_orig_test,  dataset_transf_test_pred)

print_classification_metrics(lr_aft_class_metrics_aod)

#### Results for Logistic Regression after correcting for bias (optimizing Difference in Odds)

##### Results for the training dataset


Accuracy:			 0.774
Balanced Accuracy:		 0.732
Absolute difference in odds:	 0.109
Difference in odds:		 0.036
Disparate impact:		 0.92


##### Results for the validation dataset


Accuracy:			 0.79
Balanced Accuracy:		 0.749
Absolute difference in odds:	 0.19
Difference in odds:		 -0.029
Disparate impact:		 0.873


##### Results for the testing dataset


Accuracy:			 0.733
Balanced Accuracy:		 0.705
Absolute difference in odds:	 0.137
Difference in odds:		 -0.137
Disparate impact:		 0.615


In [34]:
# Testing: Check if the Difference in Odds  has not become worse
#assert lr_aft_class_metrics_aod[13] <= lr_bef_class_metrics[13]

* Note that the results for post-processing with ROC, optimizing Disparate Impact oroptimizing Absolute Difference in Odd are very much alike (and they are often the same in other runs). Not sure if that is always the case, but for this dataset, it is. Maybe the limited number of records is causing this.

* Rather than using ROC postprocessing, aif36 provides another post-processing module, which is aimed at equalizing the odds.

* The name of that function is EqOddsPostprocessing.

### G. Post-processing the LR model with EqOddsPostprocessing<a name="subparagraph5.7"></a>

In [35]:
# Placeholders for the transformed datasets ("transformed" = transformed by post processing to equalize the odds)
dataset_transf_train_pred = dataset_orig_train.copy(deepcopy=True)
dataset_transf_valid_pred = dataset_orig_valid.copy(deepcopy=True)
dataset_transf_test_pred = dataset_orig_test.copy(deepcopy=True)

In [36]:
# Learn parameters on validation set to equalize odds and apply to create a new dataset

cpp = EqOddsPostprocessing(privileged_groups=privileged_groups,
                           unprivileged_groups=unprivileged_groups,
                           seed=1994)

cpp = cpp.fit(dataset_orig_valid, dataset_orig_valid_pred)

### H. Fairness metrics for LR model, after correcting for bias (with optimized Equalized Odds)<a name="subparagraph5.8"></a>

In [37]:
dataset_transf_train_pred = cpp.predict(dataset_orig_train_pred)
dataset_transf_valid_pred = cpp.predict(dataset_orig_valid_pred)
dataset_transf_test_pred  = cpp.predict(dataset_orig_test_pred)

In [38]:
display(Markdown("#### Results for Logistic Regression after correcting for bias (optimizing Equalized Odds)"))

lr_aft_class_metrics_eqo = compute_classification_metrics (dataset_orig_train, dataset_transf_train_pred,
                                                     dataset_orig_valid, dataset_transf_valid_pred,
                                                     dataset_orig_test,  dataset_transf_test_pred)

print_classification_metrics(lr_aft_class_metrics_eqo)

#### Results for Logistic Regression after correcting for bias (optimizing Equalized Odds)

##### Results for the training dataset


Accuracy:			 0.646
Balanced Accuracy:		 0.632
Absolute difference in odds:	 0.09
Difference in odds:		 0.09
Disparate impact:		 1.086


##### Results for the validation dataset


Accuracy:			 0.643
Balanced Accuracy:		 0.625
Absolute difference in odds:	 0.018
Difference in odds:		 -0.01
Disparate impact:		 1.01


##### Results for the testing dataset


Accuracy:			 0.711
Balanced Accuracy:		 0.726
Absolute difference in odds:	 0.021
Difference in odds:		 -0.007
Disparate impact:		 0.724


## 6. Extreme Gradient Boosting (XGB) <a name="paragraph6"></a>

### A. Build an XGB model on training data<a name="subparagraph6.1"></a>

First, tune the parameters for the XGB algorithm

* https://medium.com/all-things-ai/in-depth-parameter-tuning-for-gradient-boosting-3363992e9bae
* https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/

In [39]:
# Placeholders for the predictions on the original datasets
dataset_orig_train_pred = dataset_orig_train.copy(deepcopy=True)
dataset_orig_valid_pred = dataset_orig_valid.copy(deepcopy=True)
dataset_orig_test_pred  = dataset_orig_test.copy(deepcopy=True)

#### Tuning the parameters of XGB Classifier, using Grid Seacrh

#### The following parameters will be tuned:
* colsample_bytree:  percentage of features (columns) will be used for building each tree
* min_child_weight: minimum number of records that is required in a tree node to try to split on another feature; if the number of records falls below this minimu, the node will not be split
* learning_rate: the factor with which new trees are weighted to correct the wrong predictions of the previous tree
* n_estimators: the number of trees on which the final prediction is based
* max_depth: the maximum number of levels that a single tree can have
* gamma:  specifies the minimum loss reduction required to make a split

Parameter values will be taken close to the values used on the training set.


#### Referrences:
* https://machinelearningmastery.com/tune-learning-rate-for-gradient-boosting-with-xgboost-in-python/
* https://www.mikulskibartosz.name/xgboost-hyperparameter-tuning-in-python-using-grid-search/
* https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/

In [40]:
#from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

estimator = XGBClassifier(base_score=0.7, eval_metric="error", seed=1994)

parameters = {'colsample_bytree': [0.1, 0.5, 1],
              'min_child_weight': [5, 15, 25],
              'learning_rate': [0.001, 0.05, 0.1, 0.2, 0.30],
              'n_estimators': [10, 50, 100],
              'max_depth':[2, 4, 6],
              'gamma': [0, 0.001, 0.005]              
             }


grid_search = GridSearchCV(estimator=estimator,
                           param_grid=parameters,
                           scoring = 'balanced_accuracy',
                           n_jobs = 1,
                           cv = 10)

In [41]:
#tune_xgb_params = True
tune_xgb_params = False

if tune_xgb_params:
    start = time.time()
    grid_search.fit(X_train, y_train)
    end = time.time()
    
    xgbmod  = grid_search.best_estimator_
    minutes= (end - start) /60
    print( "It took %.2f minutes to process" % minutes)
    

In [42]:
# Copy and paste output from previous cell here (but remove missing parameter because that gives an error)
xgbmod = XGBClassifier(base_score=0.7, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5,
              enable_categorical=False, eval_metric='error', gamma=0, gpu_id=-1,
              importance_type=None, interaction_constraints='',
              learning_rate=0.3, max_delta_step=0, max_depth=2,
              min_child_weight=15, monotone_constraints='()',
              n_estimators=50, n_jobs=6, num_parallel_tree=1, predictor='auto',
              random_state=1994, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              seed=1994, subsample=1, tree_method='exact',
              validate_parameters=1, verbosity=None)

# tune_xgb_params can be set to false, so no grid search will be performed in a next run

# Alternatively, perform grid search and use:
# xgbmod  = grid_search.best_estimator_

xgbmod.fit(X_train, y_train)

XGBClassifier(base_score=0.7, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5,
              enable_categorical=False, eval_metric='error', gamma=0, gpu_id=-1,
              importance_type=None, interaction_constraints='',
              learning_rate=0.3, max_delta_step=0, max_depth=2,
              min_child_weight=15, missing=nan, monotone_constraints='()',
              n_estimators=50, n_jobs=6, num_parallel_tree=1, predictor='auto',
              random_state=1994, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              seed=1994, subsample=1, tree_method='exact',
              validate_parameters=1, verbosity=None)

### B. Fairness metrics for XGB model, before correcting for bias<a name="subparagraph6.2"></a>

In [43]:
# Prediction probs for training, validation and testing data
y_train_pred_prob = xgbmod.predict_proba(X_train)[:,fav_idx]
y_valid_pred_prob = xgbmod.predict_proba(X_valid)[:,fav_idx]
y_test_pred_prob  = xgbmod.predict_proba(X_test)[:,fav_idx]

class_thresh = 0.5

dataset_orig_train_pred.scores = y_train_pred_prob.reshape(-1,1)
dataset_orig_valid_pred.scores = y_valid_pred_prob.reshape(-1,1)
dataset_orig_test_pred.scores  = y_test_pred_prob.reshape(-1,1)

y_train_pred = np.zeros_like(dataset_orig_train_pred.labels)
y_train_pred[y_train_pred_prob >= class_thresh] = dataset_orig_train_pred.favorable_label
y_train_pred[~(y_train_pred_prob >= class_thresh)] = dataset_orig_train_pred.unfavorable_label
dataset_orig_train_pred.labels = y_train_pred

y_valid_pred = np.zeros_like(dataset_orig_valid_pred.labels)
y_valid_pred[y_valid_pred_prob >= class_thresh] = dataset_orig_valid_pred.favorable_label
y_valid_pred[~(y_valid_pred_prob >= class_thresh)] = dataset_orig_valid_pred.unfavorable_label
dataset_orig_valid_pred.labels = y_valid_pred

y_test_pred = np.zeros_like(dataset_orig_test_pred.labels)
y_test_pred[y_test_pred_prob >= class_thresh] = dataset_orig_test_pred.favorable_label
y_test_pred[~(y_test_pred_prob >= class_thresh)] = dataset_orig_test_pred.unfavorable_label
dataset_orig_test_pred.labels = y_test_pred

In [44]:
display(Markdown("### Results for Extreme Gradient Boosting before correcting for bias by post-processing"))

xgb_bef_class_metrics= compute_classification_metrics (dataset_orig_train, dataset_orig_train_pred,
                                                       dataset_orig_valid, dataset_orig_valid_pred,
                                                       dataset_orig_test,  dataset_orig_test_pred)

print_classification_metrics(xgb_bef_class_metrics)

### Results for Extreme Gradient Boosting before correcting for bias by post-processing

##### Results for the training dataset


Accuracy:			 0.799
Balanced Accuracy:		 0.722
Absolute difference in odds:	 0.106
Difference in odds:		 -0.106
Disparate impact:		 0.783


##### Results for the validation dataset


Accuracy:			 0.738
Balanced Accuracy:		 0.653
Absolute difference in odds:	 0.136
Difference in odds:		 -0.108
Disparate impact:		 0.81


##### Results for the testing dataset


Accuracy:			 0.767
Balanced Accuracy:		 0.703
Absolute difference in odds:	 0.147
Difference in odds:		 -0.147
Disparate impact:		 0.644


### C. Post-processing the XGB  model with ROC, optimizing Disparate Impact <a name="subparagraph6.3"></a>

In [45]:
# Placeholders for the predictions on the transformed datasets
dataset_transf_train_pred = dataset_orig_train.copy(deepcopy=True)
dataset_transf_valid_pred = dataset_orig_valid.copy(deepcopy=True)
dataset_transf_test_pred = dataset_orig_test.copy(deepcopy=True)

In [46]:
# Fairness metric already specified previously: Statistical Parity Difference is used.

ROC = RejectOptionClassification(unprivileged_groups=unprivileged_groups, 
                                 privileged_groups=privileged_groups, 
                                 low_class_thresh=0.01, high_class_thresh=0.99,
                                 num_class_thresh=100, num_ROC_margin=50,
                                 metric_name="Statistical parity difference",
                                 metric_ub=0.05, 
                                 metric_lb=-0.05)

ROC_di = ROC.fit(dataset_orig_valid, dataset_orig_valid_pred)

In [47]:
print("Optimal classification threshold (with fairness constraint) = %.4f" % ROC_di.classification_threshold)
print("Optimal ROC margin = %.4f" % ROC_di.ROC_margin)

Optimal classification threshold (with fairness constraint) = 0.5445
Optimal ROC margin = 0.0929


### D. Fairness metrics for XGB model, after correcting for bias (with optimized Disparate Impact)<a name="subparagraph6.4"></a>

In [48]:
# Post-process predictions
dataset_transf_train_pred = ROC_di.predict(dataset_orig_train_pred) 
dataset_transf_valid_pred = ROC_di.predict(dataset_orig_valid_pred)
dataset_transf_test_pred  = ROC_di.predict(dataset_orig_test_pred)

In [49]:
display(Markdown("### Results for Extreme Gradient Boosting after correcting for bias by post-processing - Disparate Impact"))

xgb_aft_class_metrics_di= compute_classification_metrics (dataset_orig_train, dataset_transf_train_pred,
                                                       dataset_orig_valid, dataset_transf_valid_pred,
                                                       dataset_orig_test,  dataset_transf_test_pred)

print_classification_metrics(xgb_aft_class_metrics_di)

### Results for Extreme Gradient Boosting after correcting for bias by post-processing - Disparate Impact

##### Results for the training dataset


Accuracy:			 0.78
Balanced Accuracy:		 0.748
Absolute difference in odds:	 0.108
Difference in odds:		 0.108
Disparate impact:		 1.02


##### Results for the validation dataset


Accuracy:			 0.767
Balanced Accuracy:		 0.727
Absolute difference in odds:	 0.175
Difference in odds:		 0.086
Disparate impact:		 1.042


##### Results for the testing dataset


Accuracy:			 0.722
Balanced Accuracy:		 0.672
Absolute difference in odds:	 0.071
Difference in odds:		 0.071
Disparate impact:		 0.922


In [50]:
# Testing: Check if the Disparate Impact has not become worse
#assert xgb_aft_class_metrics_di[14] >= xgb_bef_class_metrics[14]

### E. Post-processing the XGB  model with ROC, optimizing Difference in Odds<a name="subparagraph6.5"></a>

In [51]:
# Placeholders for the predictions on the transformed datasets
dataset_transf_train_pred = dataset_orig_train.copy(deepcopy=True)
dataset_transf_valid_pred = dataset_orig_valid.copy(deepcopy=True)
dataset_transf_test_pred = dataset_orig_test.copy(deepcopy=True)

In [52]:
ROC_aod = RejectOptionClassification(unprivileged_groups=unprivileged_groups, 
                                 privileged_groups=privileged_groups, 
                                 low_class_thresh=0.01, high_class_thresh=0.99,
                                 num_class_thresh=100, num_ROC_margin=50,
                                 metric_name="Average odds difference",
                                 metric_ub=0.05, 
                                 metric_lb=-0.05)

ROC_aod = ROC.fit(dataset_orig_valid, dataset_orig_valid_pred)

In [53]:
print("Optimal classification threshold (with fairness constraint) = %.4f" % ROC_aod.classification_threshold)
print("Optimal ROC margin = %.4f" % ROC_aod.ROC_margin)

Optimal classification threshold (with fairness constraint) = 0.5445
Optimal ROC margin = 0.0929


### F. Fairness metrics for XGB model, after correcting for bias (with optimized Difference in Odds)<a name="subparagraph6.6"></a>

In [54]:
# Post-process predictions
dataset_transf_train_pred = ROC_aod.predict(dataset_orig_train_pred) 
dataset_transf_valid_pred = ROC_aod.predict(dataset_orig_valid_pred)
dataset_transf_test_pred  = ROC_aod.predict(dataset_orig_test_pred)

In [55]:
display(Markdown("### Results for Extreme Gradient Boosting after correcting for bias by post-processing - Difference in Odds"))

xgb_aft_class_metrics_aod= compute_classification_metrics (dataset_orig_train, dataset_transf_train_pred,
                                                           dataset_orig_valid, dataset_transf_valid_pred,
                                                           dataset_orig_test,  dataset_transf_test_pred)

print_classification_metrics(xgb_aft_class_metrics_aod)

### Results for Extreme Gradient Boosting after correcting for bias by post-processing - Difference in Odds

##### Results for the training dataset


Accuracy:			 0.78
Balanced Accuracy:		 0.748
Absolute difference in odds:	 0.108
Difference in odds:		 0.108
Disparate impact:		 1.02


##### Results for the validation dataset


Accuracy:			 0.767
Balanced Accuracy:		 0.727
Absolute difference in odds:	 0.175
Difference in odds:		 0.086
Disparate impact:		 1.042


##### Results for the testing dataset


Accuracy:			 0.722
Balanced Accuracy:		 0.672
Absolute difference in odds:	 0.071
Difference in odds:		 0.071
Disparate impact:		 0.922


In [56]:
# Testing: Check if the Difference in Odds Impact has not become worse
# assert xgb_aft_class_metrics_aod[13] <= xgb_bef_class_metrics[13]

### G. Post-processing the XGB  model, optimizing Equalized Odds<a name="subparagraph6.7"></a>

In [57]:
# Placeholders for the transformed datasets ("transformed" = transformed by post processing to equalize the odds)
dataset_transf_train_pred = dataset_orig_train.copy(deepcopy=True)
dataset_transf_valid_pred = dataset_orig_valid.copy(deepcopy=True)
dataset_transf_test_pred = dataset_orig_test.copy(deepcopy=True)

In [58]:
# Learn parameters on validation set to equalize odds and apply to create a new dataset

cpp = EqOddsPostprocessing(privileged_groups=privileged_groups,
                           unprivileged_groups=unprivileged_groups,
                           seed=1994)

cpp = cpp.fit(dataset_orig_valid, dataset_orig_valid_pred)

### H. Fairness metrics for XGB model, after correcting for bias (with optimized Equalized Odds)<a name="subparagraph6.8"></a>

In [59]:
dataset_transf_train_pred = cpp.predict(dataset_orig_train_pred)
dataset_transf_valid_pred = cpp.predict(dataset_orig_valid_pred)
dataset_transf_test_pred  = cpp.predict(dataset_orig_test_pred)

In [60]:
display(Markdown("### Results for Extreme Gradient Boosting after correcting for bias by post-processing - Difference in Odds"))

xgb_aft_class_metrics_eqo= compute_classification_metrics (dataset_orig_train, dataset_transf_train_pred,
                                                           dataset_orig_valid, dataset_transf_valid_pred,
                                                           dataset_orig_test,  dataset_transf_test_pred)

print_classification_metrics(xgb_aft_class_metrics_eqo)

### Results for Extreme Gradient Boosting after correcting for bias by post-processing - Difference in Odds

##### Results for the training dataset


Accuracy:			 0.611
Balanced Accuracy:		 0.561
Absolute difference in odds:	 0.175
Difference in odds:		 -0.038
Disparate impact:		 0.964


##### Results for the validation dataset


Accuracy:			 0.576
Balanced Accuracy:		 0.523
Absolute difference in odds:	 0.021
Difference in odds:		 -0.004
Disparate impact:		 1.009


##### Results for the testing dataset


Accuracy:			 0.7
Balanced Accuracy:		 0.682
Absolute difference in odds:	 0.142
Difference in odds:		 -0.02
Disparate impact:		 0.776


We now have lists storing metrics for:
* Logistic regresssion, no corrections:                                          lr_bef_class_metrics
* Logistic regresssion, ROC post-processing, optimizing Disparate Impact:        lr_aft_class_metrics_di
* Logistic regresssion, ROC post-processing, optimizing Difference in Odds:      lr_aft_class_metrics_aod
* Logistic regresssion, ROC post-processing, optimizing Equalized Odds:          lr_aft_class_metrics_eqo
* Extreme Gradient Boosting, no corrections:                                     xgb_bef_class_metrics
* Extreme Gradient Boosting, ROC post-processing, optimizing Disparate Impact:   xgb_aft_class_metrics_di
* Extreme Gradient Boosting, ROC post-processing, optimizing Difference in Odds: xgb_aft_class_metrics_aod
* Extreme Gradient Boosting, ROC post-processing, optimizing Equalized Odds:     xgb_aft_class_metrics_eqo

All metrics are computed for Training, Validatiopn, and Testing dataset.

In [61]:
# !pip install tabulate

In [62]:
# from tabulate import tabulate

In [63]:
all_data = [lr_bef_class_metrics,  lr_aft_class_metrics_di,  lr_aft_class_metrics_aod, lr_aft_class_metrics_eqo,
           xgb_bef_class_metrics, xgb_aft_class_metrics_di, xgb_aft_class_metrics_aod, xgb_aft_class_metrics_eqo
           ]
rounded_list = [np.round(num, 2) for num in all_data]

df_table = pd.DataFrame (rounded_list).transpose()
new_col_names=["LR", "LR-PP (DI)",  "LR-PP (AOD)", "LR-PP (EQO)", "XGB", "XGB-PP (DI)",  "XGB-PP (AOD)", "XGB-PP (EQO)" ]

df_table = df_table.set_axis(new_col_names, axis=1)
dataset = pd.DataFrame(["Training"," "," ", " ", " ",
                       "Validation"," "," ", " ", " ",
                       "Testing"," "," ", " ", " "])
df_table.insert (0, "Dataset", dataset)

metric = ["Acc", "Bal Acc", "Abs Odds Diff", "Odds Diff", "Disp Imp"] * 3
              
df_table.insert (1, "Metric", metric)
#df_table=df_table.reset_index(drop=True)
df_table

,Dataset,Metric,LR,LR-PP (DI),LR-PP (AOD),LR-PP (EQO),XGB,XGB-PP (DI),XGB-PP (AOD),XGB-PP (EQO)
0,Training,Acc,0.78,0.78,0.77,0.65,0.80,0.78,0.78,0.61
1,,Bal Acc,0.70,0.73,0.73,0.63,0.72,0.75,0.75,0.56
2,,Abs Odds Diff,0.10,0.11,0.11,0.09,0.11,0.11,0.11,0.17
3,,Odds Diff,-0.10,0.11,0.04,0.09,-0.11,0.11,0.11,-0.04
4,,Disp Imp,0.79,1.02,0.92,1.09,0.78,1.02,1.02,0.96
5,Validation,Acc,0.76,0.80,0.79,0.64,0.74,0.77,0.77,0.58
6,,Bal Acc,0.68,0.74,0.75,0.62,0.65,0.73,0.73,0.52
7,,Abs Odds Diff,0.21,0.25,0.19,0.02,0.14,0.17,0.17,0.02
8,,Odds Diff,-0.21,0.11,-0.03,-0.01,-0.11,0.09,0.09,-0.00
9,,Disp Imp,0.73,1.02,0.87,1.01,0.81,1.04,1.04,1.01
